### Kamis openAPI

In [3]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# 과일 정의
fruits = [
    {'name': '사과', 'p_itemcategorycode': '400', 'p_itemcode': '411', 'p_kindcode': '05', 'p_graderank': '1,2'},
    {'name': '배', 'p_itemcategorycode': '400', 'p_itemcode': '412', 'p_kindcode': '01', 'p_graderank': ''},
    {'name': '감귤', 'p_itemcategorycode': '400', 'p_itemcode': '415', 'p_kindcode': '00', 'p_graderank': ''},
    {'name': '복숭아', 'p_itemcategorycode': '400', 'p_itemcode': '413', 'p_kindcode': '01', 'p_graderank': ''},
    {'name': '포도', 'p_itemcategorycode': '400', 'p_itemcode': '414', 'p_kindcode': '12,01', 'p_graderank': ''},
    {'name': '단감', 'p_itemcategorycode': '400', 'p_itemcode': '416', 'p_kindcode': '00', 'p_graderank': ''},
    {'name': '바나나', 'p_itemcategorycode': '400', 'p_itemcode': '418', 'p_kindcode': '02', 'p_graderank': ''},
    {'name': '파인애플', 'p_itemcategorycode': '400', 'p_itemcode': '420', 'p_kindcode': '02', 'p_graderank': ''},
    {'name': '망고', 'p_itemcategorycode': '400', 'p_itemcode': '428', 'p_kindcode': '00', 'p_graderank': ''},
    {'name': '오렌지', 'p_itemcategorycode': '400', 'p_itemcode': '421', 'p_kindcode': '02', 'p_graderank': ''},
    {'name': '자몽', 'p_itemcategorycode': '400', 'p_itemcode': '423', 'p_kindcode': '00', 'p_graderank': ''},
    {'name': '아보카도','p_itemcategorycode': '400', 'p_itemcode': '430', 'p_kindcode': '00', 'p_graderank': ''}
]

# Initialize a list to hold all DataFrames
all_data = []

# Loop through each fruit and fetch the data
for fruit in fruits:
    p_kindcodes = fruit['p_kindcode'].split(',')
    p_graderanks = fruit['p_graderank'].split(',') if fruit['p_graderank'] else ['']
    
    for p_kindcode in p_kindcodes:
        for p_graderank in p_graderanks:
            params = {
                'action': 'monthlySalesList',
                'p_yyyy': '2024',
                'p_period': '15',
                'p_itemcategorycode': fruit['p_itemcategorycode'],
                'p_itemcode': fruit['p_itemcode'],
                'p_kindcode': p_kindcode,
                'p_graderank': p_graderank,
                'p_countycode': '1101',
                'p_convert_kg_yn': 'Y',
                'p_cert_key': 'b1f612ae-739c-4625-bf08-ddf2d11d286c',
                'p_cert_id': '4608',
                'p_returntype': 'xml'
            }

            response = requests.get('http://www.kamis.or.kr/service/price/xml.do', params=params)

            if response.status_code == 200:
                root = ET.fromstring(response.content)
                product_data = []

                for price in root.findall('price'):
                    productclscode = price.find('productclscode').text
                    caption = price.find('caption').text

                    for item in price.findall('item'):
                        yyyy = item.find('yyyy').text
                        for month in range(1, 13):
                            month_tag = f'm{month}'
                            month_value = item.find(month_tag).text if item.find(month_tag) is not None else None
                            if month_value and month_value != '-':
                                record = {
                                    'fruit': fruit['name'],
                                    'kindcode': p_kindcode,
                                    'graderank': p_graderank,
                                    'productclscode': productclscode,
                                    'caption': caption,
                                    'index': f'{yyyy}-{month}',
                                    'value': month_value
                                }
                                product_data.append(record)

                df = pd.DataFrame(product_data)
                all_data.append(df)

# Combine all DataFrames
final_df = pd.concat(all_data, ignore_index=True)

# Create a composite column name by combining 'fruit', 'kindcode', 'graderank', 'productclscode' with 'caption'
final_df['caption'] = final_df.apply(lambda x: f"{x['fruit']}_{x['productclscode']} | {x['caption']}", axis=1)

# Pivot the DataFrame
final_df_pivot = final_df.pivot(index='index', columns='caption', values='value')

# Reset the index for easier viewing
final_df_pivot = final_df_pivot.reset_index()

# Display the final DataFrame
final_df_pivot

caption,index,감귤_01 | 소매가격 > 과일류 > 감귤 > 감귤 > 상품 > 10개,감귤_01 | 소매가격 > 과일류 > 감귤 > 감귤 > 중품 > 10개,감귤_02 | 중도매인 판매가격 > 과일류 > 감귤 > 감귤 > 상품 > 1kg,감귤_02 | 중도매인 판매가격 > 과일류 > 감귤 > 감귤 > 중품 > 1kg,단감_01 | 소매가격 > 과일류 > 단감 > 단감 > 상품 > 10개,단감_01 | 소매가격 > 과일류 > 단감 > 단감 > 중품 > 10개,단감_02 | 중도매인 판매가격 > 과일류 > 단감 > 단감 > 상품 > 1kg,단감_02 | 중도매인 판매가격 > 과일류 > 단감 > 단감 > 중품 > 1kg,망고_01 | 소매가격 > 과일류 > 망고 > 수입 > 상품 > 1개,...,파인애플_02 | 중도매인 판매가격 > 과일류 > 파인애플 > 수입 > 상품 > 1kg,파인애플_02 | 중도매인 판매가격 > 과일류 > 파인애플 > 수입 > 중품 > 1kg,포도_01 | 소매가격 > 과일류 > 포도 > 샤인머스켓 > L과 > 1kg,포도_01 | 소매가격 > 과일류 > 포도 > 샤인머스켓 > M과 > 1kg,포도_01 | 소매가격 > 과일류 > 포도 > 캠벨얼리 > L과 > 1kg,포도_01 | 소매가격 > 과일류 > 포도 > 캠벨얼리 > M과 > 1kg,포도_02 | 중도매인 판매가격 > 과일류 > 포도 > 샤인머스켓 > L과 > 1kg,포도_02 | 중도매인 판매가격 > 과일류 > 포도 > 샤인머스켓 > M과 > 1kg,포도_02 | 중도매인 판매가격 > 과일류 > 포도 > 캠벨얼리 > L과 > 1kg,포도_02 | 중도매인 판매가격 > 과일류 > 포도 > 캠벨얼리 > M과 > 1kg
0,2009-1,"2,651","1,774","1,417","1,232","10,265","7,080","3,256","2,586",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-10,"3,267","3,006","1,500","1,100","8,400","5,638","1,600","1,200",NaN,...,NaN,NaN,NaN,NaN,"4,408",NaN,NaN,NaN,"3,333","2,610"
2,2009-11,"2,104","1,820","1,257",833,"7,905","5,596","1,886","1,371",NaN,...,NaN,NaN,NaN,NaN,"4,453",NaN,NaN,NaN,"2,800","2,000"
3,2009-12,"1,687","1,400","1,265",933,"9,101","5,893","2,392","1,832",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-2,"2,773","1,800","1,457","1,207","8,246","6,781","2,517","1,903",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,2024-3,NaN,NaN,NaN,NaN,"20,626","17,957",NaN,"5,860","3,448",...,"2,668","2,448","15,357",NaN,NaN,NaN,"15,200","11,800",NaN,NaN
183,2024-4,NaN,NaN,NaN,NaN,"18,112",NaN,NaN,NaN,"3,474",...,"2,386","2,140","17,080",NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,2024-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"3,700",...,"2,549","2,305",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,2024-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"4,293",...,"2,853","2,621",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# 아보카도 포함
final_df_pivot.to_excel('과일가격(아보카도 포함)_가공전_2024_15.xlsx', index = False)

In [29]:
#final_df_pivot.to_excel('과일가격(전체)_가공전_2024_15.xlsx', index = False)

In [34]:
df = pd.read_excel('과일가격(전체)_가공전_2024_15.xlsx')

def parse_index(index_str):
    year, month = map(int, index_str.split('-'))
    # Return the first day of the month for simplicity
    return pd.Timestamp(year=year, month=month, day=1)

df['date'] = df['index'].apply(parse_index)

# Set the 'datetime' column as the index (optional)
df.set_index('date', inplace=True)

# Sort the DataFrame by the datetime index
df.sort_index(inplace=True)

df.head()
df.to_excel('과일가격(전체)_가공후_2024_15.xlsx', index = False)
#df['index']형식이 2009-1